### 安装 <a href="http://tushare.org/index.html">tushare</a> 
    pip install tushare
 

In [433]:
import tushare as ts
import matplotlib.pyplot as plt
import matplotlib.finance as mpf
import numpy as np
import talib as talib
import pandas as pd
import datetime as dt

#取所有2n天内的低点
def get_low_point(df,columns,n):
    conds = (df[columns]==df[columns])
    for i in range(0,n):
        conds = conds & (df[columns].shift(i+1)>=df[columns]) &  (df[columns].shift(-i-1)>=df[columns])  
    return df[conds] 

def get_height_point(df,col,n):
    conds = (df[columns]==df[columns])
    for i in range(0,n):
        conds = conds & (df[columns].shift(i+1)<=df[columns]) &  (df[columns].shift(-i-1)<=df[columns])  
    return df[conds]  


df = ts.get_k_data('002408', ktype='60',autype="qfq",index=False)

#RSI
df["rsi"] = talib.RSI(df['close'].values, timeperiod=9)
df["macd"],df["signal"],df["hist"] = talib.MACD(df['close'].values, fastperiod=12, slowperiod=26, signalperiod=9)

In [455]:
data = ts.get_k_data('002023', ktype='m',autype="qfq",index=False,start='2001-01-01', end='2018-05-15')
#RSI
data["rsi"] = talib.RSI(data['close'].values, timeperiod=9)
data 


,date,open,close,high,low,volume,code,rsi
0,2004-07-30,2.024,2.120,2.168,1.910,360892.53,002023,NaN
1,2004-08-31,2.120,1.875,2.167,1.723,177060.04,002023,NaN
2,2004-09-30,1.850,2.387,2.495,1.796,158067.84,002023,NaN
3,2004-10-29,2.430,2.592,2.791,2.241,177149.12,002023,NaN
4,2004-11-30,2.567,2.521,2.748,2.340,139323.08,002023,NaN
5,2004-12-31,2.527,2.357,2.713,2.289,133215.68,002023,NaN
6,2005-01-31,2.330,2.081,2.410,1.963,56365.04,002023,NaN
7,2005-02-28,2.001,2.280,2.408,2.001,39928.37,002023,NaN
8,2005-03-31,2.284,2.180,2.494,2.138,117872.96,002023,NaN
9,2005-04-29,2.188,2.147,2.542,2.087,159833.08,002023,50.747922


In [458]:
#绘制K线图
'''
fig = plt.figure(figsize=(24, 5))
ax = fig.add_subplot(1, 1, 1)
ax.set_xticks(range(0, len(data['date']), 10))
ax.set_xticklabels(data['date'][::10])
ax.legend(loc='upper left')

mpf.candlestick2_ochl(ax, data['open'], data['close'], data['high'], data['low'],width=0.5, colorup='r', colordown='green',alpha=0.6)
plt.grid()
'''
#low_Low= low[ (low<low.shift(3))&(low<=low.shift(2))& (low<=low.shift(1)) & (low<=low.shift(-1)) & (low<=low.shift(-2)) & (low<=low.shift(-3)) ] 
k_LowPoint= get_low_point(data,"low",3).dropna()
#画出低点
#plt.plot(low_Low.index.values,low_Low.values,'o',color='red')
k_LowPoint 

,date,open,close,high,low,volume,code,rsi
13,2005-08-31,1.986,1.739,2.014,1.665,535714.77,002023,41.103242
20,2006-03-31,1.739,1.655,1.783,1.563,273536.55,002023,40.480279
28,2006-11-30,2.217,1.997,2.259,1.835,311509.48,002023,47.118864
39,2007-10-31,3.536,2.750,3.563,2.500,258140.58,002023,48.874608
52,2008-11-28,1.109,1.302,1.509,1.082,485698.21,002023,34.627535
70,2010-05-31,6.842,5.890,6.921,4.866,500537.31,002023,59.803151
78,2011-01-31,8.093,6.973,8.352,6.096,628061.00,002023,53.775655
90,2012-01-31,4.727,3.707,4.779,3.419,420685.00,002023,27.813509
96,2012-07-31,4.347,3.778,4.479,3.749,497526.00,002023,32.583233
101,2012-12-31,4.002,4.571,4.591,3.744,907748.00,002023,46.077176


In [460]:
#找底背离点

dest=k_LowPoint[ (k_LowPoint["low"].shift(1)>k_LowPoint["low"]) & (k_LowPoint["rsi"].shift(1)>k_LowPoint["rsi"])  ]

dest[["date","close","low","rsi]]
 
    

NameError: name 'date' is not defined

In [ ]:
sma_10 = talib.SMA(np.array(data['close']), 10)
sma_30 = talib.SMA(np.array(data['close']), 30)

fig = plt.figure(figsize=(24, 8))
ax = fig.add_subplot(1, 1, 1)
ax.set_xticks(range(0, len(data['date']), 10))
ax.set_xticklabels(data['date'][::10])
ax.plot(sma_10, label='MA10')
ax.plot(sma_30, label='MA30')
ax.legend(loc='upper left')

#mpf.candlestick2_ochl(ax, data['open'], data['close'], data['high'], data['low'],
#                     width=0.5, colorup='r', colordown='green',
#                     alpha=0.6)
#plt.grid()


































In [446]:
data = ts.get_k_data('002023', ktype='m',autype="qfq",index=False,start='2015-01-01', end='2018-05-15')
#RSI
data["rsi"] = talib.RSI(data['close'].values, timeperiod=9)
data 

<urlopen error timed out>


,date,open,close,high,low,volume,code,rsi
0,2015-01-30,10.803,11.992,12.831,10.620,1598101.00,002023,NaN
1,2015-02-27,11.904,13.182,13.438,11.302,862062.00,002023,NaN
2,2015-03-31,13.054,17.026,18.053,12.279,3350110.87,002023,NaN
3,2015-04-30,17.076,17.135,19.005,15.980,3170511.00,002023,NaN
4,2015-05-29,17.352,27.275,30.670,16.286,7708958.00,002023,NaN
5,2015-06-30,27.680,22.424,29.542,18.681,5361146.00,002023,NaN
6,2015-07-31,22.424,21.839,24.126,13.761,7839683.00,002023,NaN
7,2015-08-31,21.107,16.632,24.691,13.464,8334378.00,002023,NaN
8,2015-09-30,16.830,15.355,17.325,13.345,4720304.00,002023,NaN
9,2015-10-30,16.088,18.592,19.850,15.662,4254273.00,002023,60.840999
